In [ ]:
!pip install xgboost scikit-learn pandas numpy


In [ ]:
print("Dataset Columns:", df.columns.tolist())

Dataset Columns: ['transaction_amount', 'transaction_date', 'transaction_channel', 'is_fraud', 'transaction_payment_mode_anonymous', 'payment_gateway_bank_anonymous', 'payer_browser_anonymous', 'payer_email_anonymous', 'payee_ip_anonymous', 'payer_mobile_anonymous', 'transaction_id_anonymous', 'payee_id_anonymous']


In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Load dataset
df = pd.read_csv("transactions_train.csv")  # Update with the actual file name

# Check column names
print("Columns in dataset:", df.columns)

# Fill missing values
df.fillna(0, inplace=True)

# Identify categorical columns and encode them
categorical_cols = [
    "payment_gateway_bank_anonymous",
    "payer_browser_anonymous",
    "payer_email_anonymous",
    "payee_ip_anonymous",
    "payer_mobile_anonymous",
    "transaction_id_anonymous",
    "payee_id_anonymous"
]
# Convert categorical columns to string type
for col in categorical_cols:
    df[col] = df[col].astype(str)  # Ensures all values are strings before encoding
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Store encoders for future use


label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Store encoders for future use

# **Make sure the target column exists**
target_column = "is_fraud"  # Update if it's named differently in your dataset

if target_column not in df.columns:
    raise KeyError(f"Target column '{target_column}' not found in dataset. Check the correct column name.")

# Separate features and target
X = df.drop(columns=[target_column])  # Features
y = df[target_column]  # Target variable (1 = Fraud, 0 = Not Fraud)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Data Preprocessing Completed ✅")


Columns in dataset: Index(['transaction_amount', 'transaction_date', 'transaction_channel',
       'is_fraud', 'transaction_payment_mode_anonymous',
       'payment_gateway_bank_anonymous', 'payer_browser_anonymous',
       'payer_email_anonymous', 'payee_ip_anonymous', 'payer_mobile_anonymous',
       'transaction_id_anonymous', 'payee_id_anonymous'],
      dtype='object')
Data Preprocessing Completed ✅


In [ ]:
# Convert 'transaction_date' to numerical (if it's a date column)
if 'transaction_date' in df.columns:
    df['transaction_date'] = pd.to_datetime(df['transaction_date'], errors='coerce')
    df['transaction_date'] = df['transaction_date'].astype('int64') // 10**9  # Convert to UNIX timestamp

# Identify object columns (categorical)
object_cols = X_train.select_dtypes(include=['object']).columns

# Apply Label Encoding to categorical columns
for col in object_cols:
    df[col] = df[col].astype(str)  # Ensure all values are strings
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

# Check again to confirm
print(X_train.dtypes)  # Should now be int, float, or category only


transaction_amount                    float64
transaction_date                       object
transaction_channel                    object
transaction_payment_mode_anonymous      int64
payment_gateway_bank_anonymous          int64
payer_browser_anonymous                 int64
payer_email_anonymous                   int64
payee_ip_anonymous                      int64
payer_mobile_anonymous                  int64
transaction_id_anonymous                int64
payee_id_anonymous                      int64
dtype: object


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Convert 'transaction_date' to numerical (UNIX timestamp)
df['transaction_date'] = pd.to_datetime(df['transaction_date'], errors='coerce')
df['transaction_date'] = df['transaction_date'].astype('int64') // 10**9  # Convert to UNIX timestamp

# Encode 'transaction_channel' (categorical)
le = LabelEncoder()
df['transaction_channel'] = le.fit_transform(df['transaction_channel'].astype(str))

# Verify Data Types
print(df.dtypes)  # Should be int or float only


transaction_amount                    float64
transaction_date                        int64
transaction_channel                     int64
is_fraud                                int64
transaction_payment_mode_anonymous      int64
payment_gateway_bank_anonymous          int64
payer_browser_anonymous                 int64
payer_email_anonymous                   int64
payee_ip_anonymous                      int64
payer_mobile_anonymous                  int64
transaction_id_anonymous                int64
payee_id_anonymous                      int64
dtype: object


In [ ]:
# Separate features and target
target_column = "is_fraud"
X = df.drop(columns=[target_column])  # Features
y = df[target_column]  # Target variable

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize XGBoost classifier
model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False
)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate performance
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Model Accuracy: {accuracy:.4f}")
print(classification_report(y_test, y_pred))


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:57:11] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✅ Model Accuracy: 0.9999
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     34584
           1       0.00      0.00      0.00         2

    accuracy                           1.00     34586
   macro avg       0.50      0.50      0.50     34586
weighted avg       1.00      1.00      1.00     34586



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import numpy as np

# Example transaction (replace with real feature values)
sample_transaction = np.array([[500.0, 1657987200, 1, 2, 3, 1, 0, 4, 5, 1000001, 2000001]])

# Make prediction
prediction = model.predict(sample_transaction)[0]
fraud_prob = model.predict_proba(sample_transaction)[0][1]  # Probability of fraud

print(f"🚀 Fraud Prediction: {'Fraud' if prediction else 'Not Fraud'}")
print(f"🔢 Fraud Probability: {fraud_prob:.4f}")


🚀 Fraud Prediction: Not Fraud
🔢 Fraud Probability: 0.0002


In [ ]:
# Get training and test accuracy
train_accuracy = model.score(X_train, y_train)
test_accuracy = model.score(X_test, y_test)

print(f"🚀 Training Accuracy: {train_accuracy:.4f}")
print(f"📉 Test Accuracy: {test_accuracy:.4f}")

# Check for overfitting
if train_accuracy > test_accuracy + 0.10:  # 10% or more difference
    print("⚠️ Overfitting detected!")
else:
    print("✅ No overfitting detected.")


🚀 Training Accuracy: 1.0000
📉 Test Accuracy: 0.9999
✅ No overfitting detected.
